<a href="https://colab.research.google.com/github/R3gm/Colab-resources/blob/main/Video_generation_FILM_with_Pause%2C_Loops%2C_and_Reverse_v1_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FiLM: Frame Interpolation for Large Motion


![google](https://research.google/static/images/share-9cd7266ef5001b20f98e01062c26189fa69ed6c784df04caf809668887fd339a.png)

| Description | Link |
| ----------- | ---- |
| 🎉 Original Repository | [![GitHub Repository](https://img.shields.io/github/stars/google-research/frame-interpolation?style=social)](https://github.com/google-research/frame-interpolation) |
| 📙 Original Colab by Stephen Young| [![PyTTI-Tools:FILM](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.sandbox.google.com/github/pytti-tools/frame-interpolation/blob/main/PyTTI_Tools_FiLM-colab.ipynb) |
| 🚀 Online inference | [![Replicate](https://replicate.com/google-research/frame-interpolation/badge)](https://replicate.com/google-research/frame-interpolation)
 |
| 🔥 Discover More Colab Notebooks | [![GitHub Repository](https://img.shields.io/badge/GitHub-Repository-black?style=flat-square&logo=github)](https://github.com/R3gm/Colab-resources/) |


Implementation of [Frame Interpolation for Large Scene Motion code repo](https://github.com/google-research/frame-interpolation). All credit goes to them. 

This notebook maintained by [Stephen Young](https://twitter.com/KyrickYoung) or SteveTheNinja#0616

Release v1.2
- fixed crash that prevented interpolation. BIG BIG shoutout to [Alec](https://twitter.com/ai_for_humans) for debugging and discovering the root cause!

Release v1.3
- Raise user warning for a space in frames_folder (instead of horribly crashing in an obscure way)
- Small UX improvement for relative and absolute frames path

Release v1.4
- Suppress double progress bars which could cause the notebook to freeze and crash on large batches

In [ ]:
#@title Check GPU Status
!nvidia-smi -L

In [ ]:
# @title Setup directory or Mount Drive
from pathlib import Path
import os

ROOT_FOLDER = "generative" # @param {type:"string"}
PROJECT_FOLDER = "frame-interpolation" # @param {type:"string"}

MOUNT_DRIVE = False #@param {type:"boolean"}

if MOUNT_DRIVE == True:
  from google.colab import drive

  MOUNTED_PATH = Path('/content/drive')
  MYDRIVE_PATH = MOUNTED_PATH / "MyDrive"
  PROJECT_PATH = MYDRIVE_PATH /  ROOT_FOLDER / PROJECT_FOLDER

  drive.mount(str(MOUNTED_PATH))

  os.makedirs(PROJECT_PATH, exist_ok=True)
else: 
  ROOT_ = Path("/content")

  PROJECT_PATH = ROOT_ /  ROOT_FOLDER / PROJECT_FOLDER

  os.makedirs(PROJECT_PATH, exist_ok=True)
  

In [ ]:
#@title Setup
!git clone https://github.com/google-research/frame-interpolation frame_interpolation
!pip install mediapy==1.0.3 gdown tensorflow==2.8.0

In [ ]:
# @title Imports

from pathlib import Path
import functools
import os
from typing import List, Sequence
import sys
import glob
import math
from base64 import b64encode

from IPython import display
from frame_interpolation.eval import interpolator as interpolator_lib
from frame_interpolation.eval import util
from absl import app
from absl import flags
from absl import logging
import mediapy as media
import natsort
import numpy as np
import tensorflow as tf
from tqdm.notebook import trange

# Setup Models

Downloading from gdrive in code is unreliable. If the code below fails you will have to copy the files into your gdrive manually.

To do so: 
1. Download the entire [pretrained_models](https://drive.google.com/drive/folders/1q8110-qp225asX3DQvZnfLfJPkCHmDpy) folder from gdrive
2. Unzip the contents on your computer
3. Upload the pretrained_models to your grdrive under the project folder. Default: MyDrive/generative/frame-interpolation/pretrained_models/

Note: the full path will be different if you've changed the root and project folders above. Path is in the structure MyDrive/ROOT_FOLDER/PROJECT_FOLDER/pretrained_models/

In [ ]:
# @title Setup Models
# @markdown Will skip if already downloaded
import gdown


models_base_path = PROJECT_PATH / "pretrained_models"

def check_models(model: str) -> bool:
    model_path = models_base_path / f"film_net/{model}/saved_model"

    return (
        (model_path / "keras_metadata.pb").exists() and
        (model_path / "saved_model.pb").exists() and 
        (model_path / "variables/variables.data-00000-of-00001").exists() and
        (model_path / "variables/variables.index").exists()
    )


def download_model(url, output_folder):
    return gdown.download_folder(url, output=output_folder, quiet=False, use_cookies=False)


# check if we've already downloaded
is_downloaded = check_models("L1") and check_models("VGG") and check_models("Style")

gdrive_url = "https://drive.google.com/drive/folders/1q8110-qp225asX3DQvZnfLfJPkCHmDpy"

if not is_downloaded:
    print("Downloading models...")
    result = download_model(gdrive_url, str(models_base_path))
    if result is None:
        msg = f"""\n
=======================================================================
 Automatic download as failed. Please upload following the steps above.
 Upload models to path: {models_base_path}
=======================================================================
        """
        raise UserWarning(msg)
else:
    print("Models already downloaded! Skipping")




# Interpolate!

In [ ]:
#@title Optional, upload your own images. `frames_folder = /content/images`
from google.colab import files
%cd /content
!mkdir images
%cd /content/images
uploaded = files.upload()
%cd /content

/content
mkdir: cannot create directory ‘images’: File exists
/content/images


Saving 01frame.jpg to 01frame.jpg
Saving 02frame.jpg to 02frame.jpg
Saving 03frame.jpg to 03frame.jpg
/content


In [21]:
#@markdown Frames png files should be in the frames folder in alphabetical order frame.png files. Files will be processed in alphabetical order
#@markdown - `01_frame.png` 
#@markdown - `02_frame.png` 
#@markdown - `03_frame.png`
#@markdown 
#@markdown Use a relative path in the project folder i.e. `my_folder`. Or an absolute path such as `/content/my_folder`
frames_folder = "/content/images" #@param{type:"string"}
#@markdown The number of times to run recursive midpoint interpolation. The number of output frames will be 2^times_to_interpolate+1. Longer is slower and smoother but takes longer to process.
times_to_interpolate = 5 #@param{type:"slider", min:0, max:10, step:1}
#@markdown Relative to video FPS. If FPS is 30, 0.3 will pause for 9 framess. Set to 0 to turn off.
transition_pause =  0.3#@param{type:"number"}
# pad start
first_frame_pause =  0.1#@param{type:"number"}
# default 30. 'Frames per second to play interpolated videos in slow motion.'
video_fps = 30 #@param{type:"number"}
# Style of the video output
#@markdown - `loop`: Create a continuous loop with the first frame. Create a smooth loop back to the start
#@markdown - `foward_reverse`: forward until the final frame. Then reverse back to the start. Warning: Doubles processing time.
#@markdown - `normal`: Start at frame 1 and progress to the final frame
video_style = "loop" #@param ['loop', 'foward_reverse', 'normal']
# select model
#@markdown - `Style`:  crisp but can produce hard edges and jitters
#@markdown - `VGG`: trade off between crisp and smooth
#@markdown - `L1`: smoother but blurrier
flim_net_model = "VGG" #@param ["VGG", "Style", "L1"]

# Add other extensions, if not either.
INPUT_EXT = ['.png', '.jpg', '.jpeg']

# THIS IS A HACK TO STOP TQDM FROM SPAMMING STDOUT
import io
import sys
from contextlib import contextmanager
@contextmanager
def capture_stdout():
    # redirect stdout and stderr to a StringIO object
    sys.stdout = sys.stderr = io.StringIO()
    try:
        yield sys.stdout
    except Exception as e:
        # if an exception occurs, print the last 100 lines of output
        output = sys.stdout.getvalue()
        lines = output.strip().split('\n')[-100:]
        print('\n'.join(lines))
        raise e
    finally:
        # reset stdout and stderr to their original values
        sys.stdout = sys.__stdout__
        sys.stderr = sys.__stderr__

# a space in frames folder causes havoc with ffmpeg
if " " in frames_folder:
    msg = f"""

Frames_folder cannot cantain a space in the name. 
Please replace the space with _ or -
    """
    raise UserWarning(msg)


# intput/output directors
frames_folder = Path(frames_folder)
if frames_folder.is_absolute():
    frames_path = frames_folder
else:
    frames_path = PROJECT_PATH / frames_folder

intermediate_output = frames_path / 'intermediate_videos'

# collect input frames
input_files = [str(f)
               for f in map(Path, sorted(glob.glob(f"{str(frames_path)}/*"))) 
               if f.is_file() and f.suffix in INPUT_EXT]

if not input_files:
    msg = f"""

No images found in folder: {frames_path}
This folder does not exist or contains no image files.
    """
    raise UserWarning(msg)

print("Interpolating", len(input_files), "images")

videos_list_file = f"{frames_path}/videos_files_list.txt"

# clear old frames
!rm "$videos_list_file"
!rm -r "$intermediate_output"
!mkdir -p "$intermediate_output"

ffmpeg_path = util.get_ffmpeg_path()
media.set_ffmpeg(ffmpeg_path)

if video_style == "loop":
    input_files.append(input_files[0])
elif video_style == "foward_reverse":
    input_files += input_files[1::-1]

frame_sets = list(zip(input_files[:-1], input_files[1:]))

# calculate padding
t_padding_frames = math.floor(video_fps * transition_pause)
s_padding_frames = math.floor(video_fps * first_frame_pause)


# load the model
print("Loading the model...")
model_path = str(PROJECT_PATH / f"pretrained_models/film_net/{flim_net_model}/saved_model")
interpolator = interpolator_lib.Interpolator(model_path)

# interpolate
for i in trange(len(frame_sets)):
    infiles = frame_sets[i]

    with capture_stdout():
        frames = list(util.interpolate_recursively_from_files(infiles, times_to_interpolate, interpolator))

    # pad the start of the animation if on the first frame
    if i == 0:
        frames = [frames[0]] * s_padding_frames + frames
 
    frames += [frames[-1]] * t_padding_frames

    # output_frames(frames, str(frames_output_path))
    output_mp4 = f'{intermediate_output}/interpolated_{i:04}.mp4'
    media.write_video(output_mp4, frames, fps=video_fps)

file_namespace = os.path.basename(os.path.normpath(frames_folder)).replace(" ", "_")
final_filename = f"{file_namespace}_{video_fps}FPS_{video_style}.mp4"
print(f'Saving final video to {final_filename}')
all_videos = [f"file '{f}'\n"
              for f in map(Path, sorted(glob.glob(f"{str(intermediate_output)}/*"))) 
              if f.is_file() and f.suffix == ".mp4"]

with open(videos_list_file, "w") as f:
    f.writelines(all_videos)

final_output_path = f"{frames_path}/{final_filename}"
!ffmpeg -y -f concat -safe 0 -i $videos_list_file -c copy "$final_output_path"

  0%|          | 0/3 [00:00<?, ?it/s]

In [22]:
#@title Play Video

mp4 = open(final_output_path,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
display.display( display.HTML(f'<video controls loop><source src="{data_url}" type="video/mp4"></video>') )
